In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import requests
import tqdm
import os, h5py
import shutil
import re
import cv2
import time
import logging

In [15]:
class TqdmLoggingHandler(logging.Handler):
    def __init__(self, level=logging.NOTSET):
        super().__init__(level)

    def emit(self, record):
        try:
            msg = self.format(record)
            tqdm.tqdm.write(msg)
            self.flush()
        except (KeyboardInterrupt, SystemExit):
            raise
        except:
            self.handleError(record)
log = logging.getLogger(__name__)
log.setLevel(logging.INFO)
log.addHandler(TqdmLoggingHandler())

In [3]:
DATA_PATH = "./data/Train/"


scale = 4
def mk_train_data(_path):
    names = os.listdir(_path)
    names = sorted(names)
    nums = names.__len__()

    data = []
    label = []

    for i in range(nums):
        name = _path + names[i]
        org_img = cv2.imread(name, cv2.IMREAD_COLOR)

        org_img = cv2.cvtColor(org_img, cv2.COLOR_BGR2YCrCb)
        
        h, w, c = org_img.shape
        
        new_height = int(h / scale)
        new_width = int(w / scale)

        lr_img = cv2.resize(org_img, (new_width, new_height), interpolation=cv2.INTER_CUBIC)
        
        hr_img = cv2.resize(lr_img, (w, h), interpolation=cv2.INTER_CUBIC)
        
        lr_img = lr_img.astype(float) / 255.
        hr_img = hr_img.astype(float) / 255.
        
        data.append(lr_img)
        label.append(hr_img)

    data = np.array(data, dtype=float)
    label = np.array(label, dtype=float)
    return data, label


    
def write_hdf5(data, labels, output_filename):

    x = data.astype(np.float32)
    y = labels.astype(np.float32)

    with h5py.File(output_filename, 'w') as h:
        h.create_dataset('data', data=x, shape=x.shape)
        h.create_dataset('label', data=y, shape=y.shape)
        
def read_training_data(file):
    with h5py.File(file, 'r') as hf:
        data = np.array(hf.get('data'))
        
        label = np.array(hf.get('label'))
        
        return data, label

if os.path.exists("./data/train.h5") is False:
    data, label = mk_train_data(DATA_PATH)
    write_hdf5(data, label, "./data/train.h5")
else:
    data, label = read_training_data(file='./data/train.h5')
print(data.shape, label.shape)

(120, 105, 140, 3) (120, 420, 560, 3)


In [4]:
def bicubic_interpolate(image,shape):
    img_resized=cv2.resize(image,shape, interpolation=cv2.INTER_CUBIC)
    return img_resized

In [5]:
def residual_block_gen(ch=64,k_s=3,st=1):
    model=tf.keras.Sequential([
        tf.keras.layers.Conv2D(ch,k_s,strides=(st,st),padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Conv2D(ch,k_s,strides=(st,st),padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(),
      ])
    return model

def Upsample_block(x, ch=256, k_s=3, st=1):
    x = tf.keras.layers.Conv2D(ch,k_s, strides=(st,st), padding='same')(x)
    x = tf.nn.depth_to_space(x, 2) # Subpixel pixelshuffler
    x = tf.keras.layers.LeakyReLU()(x)
    return x

input_lr=tf.keras.layers.Input(shape=(None,None,3))
input_conv=tf.keras.layers.Conv2D(64,9,padding='same')(input_lr)
input_conv=tf.keras.layers.LeakyReLU()(input_conv)

SRRes = input_conv

for x in range(5):
    res_output=residual_block_gen()(SRRes)
    SRRes=tf.keras.layers.Add()([SRRes,res_output])

SRRes=tf.keras.layers.Conv2D(64,9,padding='same')(SRRes)
SRRes=tf.keras.layers.BatchNormalization()(SRRes)

SRRes=tf.keras.layers.Add()([SRRes,input_conv])

SRRes=Upsample_block(SRRes)
SRRes=Upsample_block(SRRes)

output_sr=tf.keras.layers.Conv2D(3,9,activation='tanh',padding='same')(SRRes)

SRResnet=tf.keras.models.Model(input_lr,output_sr)

In [9]:
SRResnet.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 6 15616       input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, None, None, 6 0           conv2d[0][0]                     
__________________________________________________________________________________________________
sequential (Sequential)         (None, None, None, 6 74368       leaky_re_lu[0][0]                
______________________________________________________________________________________________

In [22]:
def residual_block_disc(ch=64,k_s=3,st=1):
    model=tf.keras.Sequential([
        tf.keras.layers.Conv2D(ch,k_s,strides=(st,st),padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(),
      ])
    return model

input_lr=tf.keras.layers.Input(shape=(420, 560,3))
input_conv=tf.keras.layers.Conv2D(64,3,padding='same')(input_lr)
input_conv=tf.keras.layers.LeakyReLU()(input_conv)

channel_nums=[64,128,128,256,256,512,512]
stride_sizes=[2,1,2,1,2,1,2]

disc=input_conv

for x in range(7):
    disc=residual_block_disc(ch=channel_nums[x],st=stride_sizes[x])(disc)

disc=tf.keras.layers.Flatten()(disc)

disc=tf.keras.layers.Dense(1024)(disc)
disc=tf.keras.layers.LeakyReLU()(disc)

disc_output=tf.keras.layers.Dense(1,activation='sigmoid')(disc)

discriminator=tf.keras.models.Model(input_lr,disc_output)

In [23]:
discriminator.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 420, 560, 3)]     0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 420, 560, 64)      1792      
_________________________________________________________________
leaky_re_lu_22 (LeakyReLU)   (None, 420, 560, 64)      0         
_________________________________________________________________
sequential_12 (Sequential)   (None, 210, 280, 64)      37184     
_________________________________________________________________
sequential_13 (Sequential)   (None, 210, 280, 128)     74368     
_________________________________________________________________
sequential_14 (Sequential)   (None, 105, 140, 128)     148096    
_________________________________________________________________
sequential_15 (Sequential)   (None, 105, 140, 256)     2961

In [24]:
def PSNR(y_true,y_pred):
    mse=tf.reduce_mean( (y_true - y_pred) ** 2 )
    return 20 * log10(1 / (mse ** 0.5))

def log10(x):
    numerator = tf.math.log(x)
    denominator = tf.math.log(tf.constant(10, dtype=numerator.dtype))
    return numerator / denominator

def pixel_MSE(y_true,y_pred):
    return tf.reduce_mean( (y_true - y_pred) ** 2 )

In [25]:
VGG19=tf.keras.applications.VGG19(weights='imagenet',include_top=False,input_shape=(420, 560,3))

VGG_i,VGG_j=2,2

def VGG_loss(y_hr,y_sr,i_m=2,j_m=2):
    i,j=0,0
    accumulated_loss=0.0
    for l in VGG19.layers:
        cl_name=l.__class__.__name__
        if cl_name=='Conv2D':
            j+=1
        if cl_name=='MaxPooling2D':
            i+=1
            j=0
        if i==i_m and j==j_m:
            break

    y_hr=l(y_hr)
    y_sr=l(y_sr)
    if cl_name=='Conv2D':
        accumulated_loss+=tf.reduce_mean((y_hr-y_sr)**2) * 0.006

    return accumulated_loss


def VGG_loss_intuitive(y_true,y_pred):
    accumulated_loss=0.0
    for l in VGG19.layers:
        y_true=l(y_true)
        y_pred=l(y_pred)
        accumulated_loss+=tf.reduce_mean((y_true-y_pred)**2) * 0.006
    return accumulated_loss

In [26]:
generator_optimizer=tf.keras.optimizers.SGD(0.0001)
discriminator_optimizer=tf.keras.optimizers.SGD(0.0001)

adv_ratio=0.001
evaluate=['PSNR']
# MSE
loss_func,adv_learning = pixel_MSE,False
# VGG2.2
loss_func,adv_learning = lambda y_hr,y_sr:VGG_loss(y_hr,y_sr,i_m=2,j_m=2),False
# VGG 5.4
loss_func,adv_learning = lambda y_hr,y_sr:VGG_loss(y_hr,y_sr,i_m=5,j_m=4),False
# SRGAN-MSE
loss_func,adv_learning = pixel_MSE,True
# SRGAN-VGG 2.2
loss_func,adv_learning = lambda y_hr,y_sr:VGG_loss(y_hr,y_sr,i_m=2,j_m=2),True
# SRGAN-VGG 5.4
loss_func,adv_learning = lambda y_hr,y_sr:VGG_loss(y_hr,y_sr,i_m=5,j_m=4),True

#Real loss
loss_func,adv_learning = lambda y_hr,y_sr:VGG_loss(y_hr,y_sr,i_m=5,j_m=4),True

In [27]:
cross_entropy = tf.keras.losses.BinaryCrossentropy()
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

@tf.function()
def train_step(lr_data, hr_data,loss_func=pixel_MSE,adv_learning=True,evaluate=['PSNR'],adv_ratio=0.001):
    logs = {}
    gen_loss,disc_loss=0,0

    low_resolution, high_resolution = lr_data, hr_data
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        super_resolution = SRResnet(low_resolution, training=True)
        gen_loss=loss_func(high_resolution,super_resolution)
        
        logs['reconstruction']=gen_loss
        
        if adv_learning:
            real_output = discriminator(high_resolution, training=True)
            fake_output = discriminator(super_resolution, training=True)

            adv_loss_g = generator_loss(fake_output) * adv_ratio
            gen_loss += adv_loss_g

            disc_loss = discriminator_loss(real_output, fake_output)
            logs['adv_g']=adv_loss_g
            logs['adv_d']=disc_loss


    gradients_of_generator = gen_tape.gradient(gen_loss, SRResnet.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, SRResnet.trainable_variables))

    if adv_learning:
        gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
        discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    for x in evaluate:
        if x=='PSNR':
            logs[x]=PSNR(high_resolution,super_resolution)

    return logs

In [28]:
data.shape

(120, 105, 140, 3)

In [29]:
for x in range(10):
    lr_data, hr_data = data, label 
    for lr, hr in tqdm.tqdm(zip(data, label)):
        lr = tf.expand_dims(lr, axis=0)
        hr = tf.expand_dims(hr, axis=0)
        logs=train_step(lr,hr,loss_func,adv_learning,evaluate,adv_ratio)
        for k in logs.keys():
            print(k,':',logs[k],end='  ')
    print()

1it [07:21, 441.54s/it]

reconstruction : tf.Tensor(0.0, shape=(), dtype=float32)  adv_g : tf.Tensor(nan, shape=(), dtype=float32)  adv_d : tf.Tensor(nan, shape=(), dtype=float32)  PSNR : tf.Tensor(nan, shape=(), dtype=float32)  reconstruction : 

2it [07:21, 181.94s/it]

tf.Tensor(0.0, shape=(), dtype=float32)  adv_g : tf.Tensor(nan, shape=(), dtype=float32)  adv_d : tf.Tensor(nan, shape=(), dtype=float32)  PSNR : tf.Tensor(nan, shape=(), dtype=float32)  reconstruction : 

3it [07:21, 98.97s/it] 

...

tf.Tensor(0.0, shape=(), dtype=float32)  adv_g : tf.Tensor(nan, shape=(), dtype=float32)  adv_d : tf.Tensor(nan, shape=(), dtype=float32)  PSNR : tf.Tensor(nan, shape=(), dtype=float32)  reconstruction : 

119it [00:26,  4.47it/s]

tf.Tensor(0.0, shape=(), dtype=float32)  adv_g : tf.Tensor(nan, shape=(), dtype=float32)  adv_d : tf.Tensor(nan, shape=(), dtype=float32)  PSNR : tf.Tensor(nan, shape=(), dtype=float32)  reconstruction : 

120it [00:26,  4.47it/s]

tf.Tensor(0.0, shape=(), dtype=float32)  adv_g : tf.Tensor(nan, shape=(), dtype=float32)  adv_d : tf.Tensor(nan, shape=(), dtype=float32)  PSNR : tf.Tensor(nan, shape=(), dtype=float32)  
